# Imports

In [1]:
#imports
import sys
import pandas as pd
import random
import os
import json
import pprint

from frameit.corpus import Corpus
from frameit.utils import *
from frameit.drop_gold_from_train import dropGold
from frameit.EvalAFrame import evalFrame

pp = pprint.PrettyPrinter(indent=4)
os.environ["KERAS_BACKEND"] = "tensorflow"

/Users/jengelwork/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Introduction

These notebooks are set up to train a frame that can detect sentences about meals. Feel free to replace the default data files and modify the code as needed to adapt these notebooks to your purposes

# Loading the corpus

Your corpus should be in a .csv file, and the text to be used as training should be in a column titled "text", with each data point on a separate line. If you are planning on using a gold set, you should also have an "Index" column with id numbers for each row. 

In [2]:
corpus_file = "../resources/happy_moment_corpus_small.csv"

# Optional: set up a gold set and drop it from the training file

If you have a gold set of positive and negative examples in an XML file, you can drop those examples from
the training data with the following code.

See the instructions in docs/evaluation.rst for more information on formatting data for the evaluation script.

In [3]:
#If your positive and negative gold examples are in the same file, you can
#pass that file to both parameters–positive examples in the negative file and negative examples in the positive file
#will simply be ignored
positive_example_file = '../resources/meal_gold_set.xml'
negative_example_file = '../resources/meal_gold_set.xml'
#Note: for the default data set which is abnormally small, we use a sample size of 5. For your own purposes,
#we recommend using at least 100 examples.
corpus_file, gold_file = dropGold(corpus_file, positive_example_file, negative_example_file, prefix="../resources/")

Seed is  22
Indexing data file
Recompiling training file
file_prefix  
Writing gold set to xml file
Done


## Build the corpus

In [4]:
#Corpus data should have one sentence per line in a column titled "text". Any other columns will be ignored
#When loading a new corpus for the first time, set build_index to True to create indices necessary to process the data.
#Otherwise, this step can be safely skipped to significantly speed up runtime by setting build_index to False
corpus = Corpus(corpus_file, build_index=True)

init Corpus
Parsing the Semafor data... 
Parsing the DeepSRL data... 
Creating Utterances...


/Users/jengelwork/Documents/FrameItExport/frameit/frameit/corpus.py:54: UserWarning: No FrameNet data found for the corpus.
  warnings.warn('No FrameNet data found for the corpus.')
/Users/jengelwork/Documents/FrameItExport/frameit/frameit/corpus.py:64: UserWarning: No ProbBank data found for the corpus.
  warnings.warn('No ProbBank data found for the corpus.')
  0%|          | 0/1989 [00:00<?, ?it/s]

Loading the en model

    Info about spaCy

    spaCy version      2.0.12         
    Location           /Users/jengelwork/miniconda3/lib/python3.6/site-packages/spacy
    Platform           Darwin-17.7.0-x86_64-i386-64bit
    Python version     3.6.1          
    Models             en_core_web_md, en_core_web_lg, fr, en, fr_core_news_sm


    Info about spaCy

    spaCy version      2.0.12         
    Location           /Users/jengelwork/miniconda3/lib/python3.6/site-packages/spacy
    Platform           Darwin-17.7.0-x86_64-i386-64bit
    Python version     3.6.1          
    Models             en_core_web_md, en_core_web_lg, fr, en, fr_core_news_sm



 34%|███▍      | 677/1989 [00:00<00:00, 6766.73it/s]

Indexing corpus...


  0%|          | 8/1989 [00:00<00:24, 79.47it/s]

Indexing corpus by lemma...


100%|██████████| 1989/1989 [00:22<00:00, 88.63it/s]


# Constructing a positive set for training

## A starting point for the positive set

In [5]:
# positive_strings = ['example', 'strings', 'that would be in the', 'positive', 'sentences', 'for', 'the intent',
#                    'that you want', 'to extract']
positive_strings = ['breakfast', 'brunch', 'lunch', 'dinner']
positive_utterances = build_positive_set(corpus, positive_strings)
#Note: for exact matches of the strings, use the above function call to build_positive_set(). 
#To also include matches of all tenses and plural/singular forms of all words in the string, add_lemmas_to_set()
lemma_strings = ['restaurant', 'cafe']
positive_utterances = add_lemmas_to_set(corpus, lemma_strings, existing_set=positive_utterances)
negative_set = set()

There are 127 relevant messages in the corpus
There are 133 relevant messages in the corpus


## Optional: expand using hypernyms

In [6]:
#A hypernym h of a word w is a more generic term that includes w as part of its semantic field. 
#For example, "bird" is a hypernym of "pigeon", "eagle", "falcon", etc. "Animal" is a hypernym of "bird".

#Expanding with hypernyms may not always be appropriate. You may also want to use a different set of terms than 
#the full list of positive_strings defined earlier

positive_utterances = expand_with_hypernym(positive_utterances, positive_strings, corpus)

Number of strings for which no hypernyms were found  0
breakfast
dinner
brunch
lunch
There are 133 relevant messages in the corpus


## Sample sentences to check positive set accuracy

In [7]:
for a in random.sample(positive_utterances, 20):
    print(a.text)
    print()

My daughter and I made dinner together and had meaningful conversation.

I had time to get lunch with a friend I had not seen.

Cooked myself a wholesome breakfast and drove to work while I watched the sun rise.

I had lunch with my girlfriend at a nice restaurant.

I had a yummy lunch.

We had my favorite meal for dinner.

Today on my way to pick up dinner, I heard my current favorite song twice.

My son ate dinner with me tonight.

I made a dinner of baked ribs and potoatoes with onions.

When I grilled dinner for the family and they complimented me

I enjoy making new recipes for dinner.

I went to my favorite cafe during lunch

Got to see my son for lunch.

The pizza I had for dinner last night was delicious.

When my partner came home from work, it was great to see him and get to eat dinner and laugh together.

The fish I caught in the net displayed a fine glimmer in the sun, and became my lunch!

My daughter ate all of her dinner including her vegetables.

Invited my brother and 

## Trimming the positive set

In [8]:
#To remove bad examples from the positive set. Also creates a negative set that can optionally be used
#Note: there may not necessarily be any bad examples to trim, in which case you should skip this step.
remove_list = ['strings', 'that occur', 'in the positive set', 'that correspond', 'to examples',
               'that are not positive']
positive_utterances, negative_set = trim_examples(positive_utterances, remove_list)

There are 133 relevant messages in the corpus


# Save the data for training

## Specify hyperparameters for training

If you like, you can customize hyperparameters for the training function. Otherwise, the function will be run with default values

In [9]:
scale_to=700
epochs=40
batch_size=1400
reg_param=0.02

In [10]:
# Give the frame a name and save it to a file
frame_info_filename = 'frame_training_info.json'
frame_name = "Your Frame Name"
save_frame_training_info_to_file(frame_name, corpus_file, positive_utterances, negative_set,
                                scale_to, epochs, batch_size, reg_param, frame_info_filename, gold_file)

Saved info with filename frame_training_info.json.


Stop here and use *Generic lambda_rule attribute exploration.ipynb* and/or *Generic machine-learning attribute exploration.ipynb* if you would like to train attributes for entity-extraction to be used with this frame. When you've collected the necessary data for attributes that you want to train, proceed to the *Train frame* notebook